In [ ]:
import pandas as pd
import re
import numpy as np
from tqdm import tqdm
from nltk import word_tokenize
from fuzzywuzzy import fuzz
from collections import Counter
from string import punctuation
from string import digits
from stop_words import get_stop_words
from nltk.corpus import stopwords
from fuzzywuzzy import process

In [23]:
punct = punctuation+'©«»—…“”*№– ' + digits
stop_words_list = get_stop_words('ru')
stop_words_list2 = stopwords.words('russian')
newlist = stop_words_list + stop_words_list2
for word in ["свой"]:
    newlist.append(word)
newlist_set = set(newlist)

In [ ]:
df = pd.read_csv('result_tar.gz', compression='gzip', sep='\t', error_bad_lines=False)

In [ ]:
df = df.drop(["url", "video_url", "source_name", "author_name"], axis=1)
df = df.dropna()
df = df.drop_duplicates()
df = df.reset_index(drop=True)

In [7]:
def which_language (text):
    if bool(re.search('[\u0400-\u04FF]', text)) == True:
        return 'russian'
    else:
        return 'english'
    
lang=[]
for i,text in enumerate(df['result.tsv'].values): 
    lang.append(which_language(str(text)))
    
df['Language'] = lang
grouped=df.groupby('Language')
for name, group in grouped: 
    if name == 'english': 
        english = group.drop('Language',axis=1) 
    else: 
        russian = group.drop('Language',axis=1)


In [8]:
unique_leagues = russian['league_name_in_russian'].unique().tolist()
len(unique_leagues)

288

In [9]:
unique_teams1 = russian['team_of_season_1_name_in_russian'].unique().tolist()
len(unique_teams1)

3773

In [10]:
unique_teams2 = russian['team_of_season_2_name_in_russian'].unique().tolist()
len(unique_teams2)

3730

In [11]:
unique_teams = unique_teams1 + unique_teams2
unique_teams_set = set(unique_teams)
len(unique_teams_set)

4422

In [13]:
russian = pd.read_csv('russian_lists_and_directmatch.csv')

In [38]:
russian = russian[russian['result.tsv'].map(len) > 30]

In [39]:
russian = russian.reset_index(drop=True)

In [41]:
russian.shape

(524733, 10)

In [49]:
russian.head(3)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian,list_direct_match_for_teams1,list_direct_match_for_teams2
0,Сборная Турции продолжает свою дорогу на Чемпи...,1.65,"- 1,5",Олимп,2019-06-02 18:00:00,Международные товарищеские матчи,Турция,Узбекистан,0.0,1.0
1,Двадцать девятого числа состоится один весьма ...,1.52,"– тотал меньше 2,5",Winline,2019-06-28 23:20:00,Кубок Америки,Колумбия,Чили,0.0,0.0
2,АЗ Алкмар В первом матче голландский клуб созд...,1.85,АЗ Алкмар забьет в обоих таймах,Лига Ставок,2019-08-15 18:30:00,Лига Европы УЕФА,АЗ Алкмаар,Мариуполь,0.0,0.0


In [55]:
subset_russian = russian.sample(n=10000)
subset_russian.head(5)


,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian,list_direct_match_for_teams1,list_direct_match_for_teams2
75687,"Для Реала этот матч - чистая формальность, для...",1.38,ИТБ 1 0.5,Марафон,2019-01-16 20:30:00,Кубок Испании,Леганес,Реал Мадрид,1.0,0.0
489754,"Ну и вот такую х....ю поставим, авось залетит,...",3.40,ФОРА1 (-1),1xСтавка,2019-08-29 17:15:00,Лига Европы УЕФА,Спартак,Брага,0.0,0.0
445084,Партизан встречается с голландским АЗ. Гости п...,2.04,Ф1 (0),Леон,2019-09-19 19:00:00,Лига Европы УЕФА,Партизан,АЗ Алкмаар,0.0,0.0
108625,"По расчетам там, процент вероятность на то что...",1.23,ИТМ 1 2.5,Winline,2019-08-10 16:00:00,Премьер-Лига,Краснодар,Рубин,0.0,0.0
230916,Футбольная суббота - множество матчей пройдут ...,2.17,П1,1xСтавка,2019-09-14 14:00:00,Премьер-лига,Брайтон,Бернли,1.0,1.0


In [60]:
subset_russian = subset_russian.reset_index(drop=True)
subset_russian.head(5)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian,list_direct_match_for_teams1,list_direct_match_for_teams2
0,"Для Реала этот матч - чистая формальность, для...",1.38,ИТБ 1 0.5,Марафон,2019-01-16 20:30:00,Кубок Испании,Леганес,Реал Мадрид,1.0,0.0
1,"Ну и вот такую х....ю поставим, авось залетит,...",3.40,ФОРА1 (-1),1xСтавка,2019-08-29 17:15:00,Лига Европы УЕФА,Спартак,Брага,0.0,0.0
2,Партизан встречается с голландским АЗ. Гости п...,2.04,Ф1 (0),Леон,2019-09-19 19:00:00,Лига Европы УЕФА,Партизан,АЗ Алкмаар,0.0,0.0
3,"По расчетам там, процент вероятность на то что...",1.23,ИТМ 1 2.5,Winline,2019-08-10 16:00:00,Премьер-Лига,Краснодар,Рубин,0.0,0.0
4,Футбольная суббота - множество матчей пройдут ...,2.17,П1,1xСтавка,2019-09-14 14:00:00,Премьер-лига,Брайтон,Бернли,1.0,1.0


### Команда 1

In [ ]:
for i, text in tqdm(enumerate(subset_russian['result.tsv'].values)):
    if subset_russian['list_direct_match_for_teams1'][i] == 1:
        continue
    else:
        #print(i)
        capital_words_ru = re.findall(r'([А-Я][а-я]+)', text)
        quotes = re.findall(r'\"(.+?)\"', text)
        quotes2 = re.findall(r'«(.+?)»', text)
        quotes3 = re.findall(r'\'(.+?)\'', text)
        search_words = capital_words_ru + quotes + quotes2 + quotes3
        search_words = set(search_words)
        good_words = [word for word in search_words if word.lower() not in newlist_set] 
        #print(good_words)
    
        for word in good_words:
            for team in unique_teams:
                if fuzz.ratio(word, team) > 75 and subset_russian['team_of_season_1_name_in_russian'][i] == word:
                    #print(russian['team_of_season_2_name_in_russian'][i], word, team, fuzz.ratio(word, team))
                    subset_russian['list_direct_match_for_teams1'][i] = 1

                    break


In [65]:
success = subset_russian[subset_russian['list_direct_match_for_teams1'] == 1].shape[0]
print(success)
all_vals = subset_russian['list_direct_match_for_teams1'].shape[0]
print(all_vals)
baseline_team1 = success / all_vals
print(baseline_team1)

4413
10000
0.4413


### Команда 2 

In [ ]:
for i, text in tqdm(enumerate(subset_russian['result.tsv'].values)):
    if subset_russian['list_direct_match_for_teams2'][i] == 1:
        continue
    else:
        #print(i)
        capital_words_ru = re.findall(r'([А-Я][а-я]+)', text)
        quotes = re.findall(r'\"(.+?)\"', text)
        quotes2 = re.findall(r'«(.+?)»', text)
        quotes3 = re.findall(r'\'(.+?)\'', text)
        search_words = capital_words_ru + quotes + quotes2 + quotes3
        search_words = set(search_words)
        good_words = [word for word in search_words if word.lower() not in newlist_set] 
        #print(good_words)
    
        for word in good_words:
            for team in unique_teams:
                if fuzz.ratio(word, team) > 75 and subset_russian['team_of_season_2_name_in_russian'][i] == word:
                    #print(russian['team_of_season_2_name_in_russian'][i], word, team, fuzz.ratio(word, team))
                    subset_russian['list_direct_match_for_teams2'][i] = 1

                    break


In [66]:
success = subset_russian[subset_russian['list_direct_match_for_teams2'] == 1].shape[0]
print(success)
all_vals = subset_russian['list_direct_match_for_teams2'].shape[0]
print(all_vals)
baseline_team2 = success / all_vals
print(baseline_team2)

4144
10000
0.4144


In [64]:
subset_russian.to_csv(r'C:\Users\User\Desktop\project\fuzzylists0106.csv', index = False)